### 접근
- $O(N \log N)$ 까지 풀 수 있는 문제이다.
- 어떤 $A, B$ XOR 했을 때 그 값이 가장 크다는 것은 $A + B$ 가 크거나, $A \land B$ 가 가장 적어야 한다.
- 실제로 정렬해서 풀 순 없지만, 정렬 했을 때 맨 처음과 맨 마지막을 XOR 한 것이 항상 최적해일까?
  - 우선 조금만 생각해도 아닌것을 알 수 있다. 반례: 1 3 12
  - 게다가 XOR 합을 최대로 만드는 두 수는 $L[i]$ 단일이 아닌, 특정 구간을 XOR 합 때에만 나오는 수 일 수 있다.
- 비트 집합의 관점에서 생각해보면, 수열을 적당히 xor해서 가능한 가장 길고 -> True인 bit가 많아야 이득이라고 볼 수 있다.
  - 어떻게 활용해야할진 모르겠다.
- 우선 max, xor를 노드로 두는 세그트리를 만들어 보기로 했다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

from dataclasses import dataclass
@dataclass
class Node :
  xor: int = 0
  max: int = 0

class ST : 
  def __init__(self, L):
    self._size = _size = 1 << (len(L) - 1).bit_length()
    self.seg = [Node() for _ in range(2 * _size)]
    self.seg[_size:_size + len(L)] = [Node(xor = v, max = v) for v in L]
    self.init()

  def merge(self, l: Node, r: Node):
    ret = Node()
    ret.xor = l.xor ^ r.xor
    ret.max = max(l.max, r.max, ret.xor)
    return ret

  def init(self):
    for i in reversed(range(self._size)) :
      L = self.seg[i * 2]
      R = self.seg[i * 2 + 1]
      self.seg[i] = self.merge(L, R)
      
  def __getitem__(self, i) -> Node:
    return self.seg[i + self._size]

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  st = ST(L)
  print(st.seg[0].max)

for _ in range(int(input())) :
  sol()

- WA를 받았다. 그도 그럴것이, 루트 노드의 값이라 함은 모든 구간을 전부 따졌을 때에만 가능하다. \
만약 최적해가 [2, N-2] 쯤에서 존재한다고 하면 루트 노드엔 그것이 반영되지 않는다는 뜻이다.\
적절히 쿼리를 해야 하는 문제인지 잘 모르겠다. 애초에 세그가 잘못됐을지도.

- $L$ 에서 가장 큰 수의 비트집합 크기가 $N$ 이라고 했을 때, 최적해의 upper bound는 $2^N - 1$ 이라는 것을 발견했다. 즉, 해당 수를 이진수로 바꿨을 때의 bitmask 보다 작다는 것이다.
- 그렇다면 $L$ 에서 비트 집합의 크기가 $N$ 인 것들을 $P$ 라고 하고, 그 좌/우 구간을 살피면서 그 구간의 xor합이 $p$ 의 complementary bit가 가장 많은 수를 찾는다는 접근은 어떨까?
  - 문제는, 구간을 어떻게 잡아야 하냐는 것이다. 구간의 크기는 최대 $N-1$ 일 수 있는데, 최적해를 만드는 complementary를 어떻게 찾아야 한다는 뜻일까?
  - 연속하는 수열을 잡아야 한다는 것을 다시 상기했더니 느낌이 왔다. 구간을 1씩 늘려나가면서 complementary를 찾는 것이다.
    - 이때의 최악케이스는 $\frac {N}{2}$ 개의 pivot이 한 칸 걸러 존재할 때, 각 pivot이 양쪽 1개씩의 구간, 즉 2개의 쿼리를 발생시킬 때일 것이다.\
    쿼리는 $O(\log N)$ 이므로, $O(N \log N)$ 이다. 할만한 풀이같다.
    - 사실 양쪽 끝이 pivot일 때 나머지 구간을 2번씩 살펴보는 경우가 최악인 것 같다. 그럼에도 $O(2N \log N) = O(N \log N)$ 이다.
  - 반례가 있다. pivot을 짝수개 통과했을 때 최적해가 되는 경우가 있다. ex) 001000, 100000, 100000, 110111